In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
df = pd.DataFrame()

In [3]:
for(root, dirs, files) in os.walk(r"C:\MyStuff\Courses\Spring22\Project\Mouse-Dynamics-Challenge\training_files"):
    user = root.split("\\")[-1]
    
    for session in files:
        
        file_df = pd.read_csv(root + "/" + session)
        file_df['user_id'] = user
        file_df['session_id'] = session
        df = pd.concat([df, file_df], ignore_index=True)

In [4]:
for(root, dirs, files) in os.walk(r"C:\MyStuff\Courses\Spring22\Project\Mouse-Dynamics-Challenge\test_files"):
    user = root.split("\\")[-1]
    
    for session in files:
        
        file_df = pd.read_csv(root + "/" + session)
        file_df['user_id'] = user
        file_df['session_id'] = session
        df = pd.concat([df, file_df], ignore_index=True)

In [5]:
features = []
for session in df.groupby('session_id'):
    duration = session[1]['record timestamp'].iloc[-1] - session[1]['record timestamp'].iloc[0]
    action = len(session[1])
    x_diffs = [session[1].iloc[i+1]['x'] - session[1].iloc[i]['x'] for i in range(action-1)]
    y_diffs = [session[1].iloc[i+1]['y'] - session[1].iloc[i]['y'] for i in range(action-1)]
    avg_speed = sum([((xd**2 + yd**2)**0.5) / duration for xd, yd in zip(x_diffs, y_diffs)]) / action
    features.append([duration, action, avg_speed])

In [ ]:
proc_data = pd.DataFrame(features, columns=['duration', 'action', 'avg_speed'])

In [ ]:
anomalous_criteria = {
    'action': 3000,
    'duration': 950,
    'abnormal_movement': True,
    'unusual_click': True
}

In [ ]:
anomalous_sessions = (
    (proc_data['action'] < anomalous_criteria['action']) |
    (proc_data['duration'] < anomalous_criteria['duration']) 
)

In [ ]:
df['data'] = df['button'].astype(str) + ' ' + df['state'].astype(str)

In [ ]:
text = ''
dat =[]
for _, session in df.groupby('session_id'):
    text += session['data'].str.cat(sep=' ') + ' ' 
    dat.append(text)
    text = ''

In [ ]:
preprocessed_data = pd.DataFrame(dat, columns=['mouse_data'])

In [ ]:
preprocessed_data['label'] = anomalous_sessions.astype(int)

In [ ]:
preprocessed_data.to_csv('C:\MyStuff\Courses\Spring22\Project\Final\labeled_data_normal.csv', index=False)